In [2]:
#Load Neccessary for web scraping
from selenium import webdriver
from bs4 import BeautifulSoup
import time
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
service = ChromeService(executable_path=ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

from selenium.webdriver.common.by import By
#driver.get("https://cec.gmu.edu/about/meet-our-faculty?page=0")

[WDM] - Downloading: 100%|████████████████████████████████████████████████████████| 6.29M/6.29M [00:00<00:00, 77.6MB/s]


In [4]:
#Collects the name and profile link for each person on the CEC faculty list
profiles_final = []
names = []
for i in range(0,25):
    #Sleep to avoide bot detection
    time.sleep(3)
    #Load each new page since each page only holds 10 names
    driver.get(f"https://cec.gmu.edu/about/meet-our-faculty?page={i}")
    src = driver.page_source  
    # Now using beautiful soup
    soup = BeautifulSoup(src, 'html.parser')
    soup = soup.find("ul", {"class": "profile-list"})
    urls = []
    # traverse paragraphs from soup
    for link in soup.find_all("a"):
        data = link.get('href')
        name = link.get_text()
        #Add name and link to lists
        urls.append(data)
        names.append(name)

    mystring = 'https://cec.gmu.edu'
    profiles_final = profiles_final + [mystring + s for s in urls]
 

['https://cec.gmu.edu/profiles/scarabal', 'https://cec.gmu.edu/profiles/ladelman', 'https://cec.gmu.edu/node/8046', 'https://cec.gmu.edu/profiles/pahmadi', 'https://cec.gmu.edu/profiles/malbanes', 'https://cec.gmu.edu/profiles/jallbeck', 'https://cec.gmu.edu/profiles/mamirida', 'https://cec.gmu.edu/node/7031', 'https://cec.gmu.edu/profiles/pammann', 'https://cec.gmu.edu/profiles/antonis', 'https://cec.gmu.edu/profiles/kandrea', 'https://cec.gmu.edu/node/7976', 'https://cec.gmu.edu/profiles/ascoli', 'https://cec.gmu.edu/node/6841', 'https://cec.gmu.edu/profiles/jaurerba', 'https://cec.gmu.edu/profiles/jauffret', 'https://cec.gmu.edu/profiles/iavramo2', 'https://cec.gmu.edu/profiles/aydin', 'https://cec.gmu.edu/profiles/pbagchi', 'https://cec.gmu.edu/profiles/foteini', 'https://cec.gmu.edu/profiles/jbaldo', 'https://cec.gmu.edu/profiles/ball', 'https://cec.gmu.edu/profiles/dbarbara', 'https://cec.gmu.edu/profiles/abehesh', 'https://cec.gmu.edu/node/6946', 'https://cec.gmu.edu/profiles/fb

In [5]:
#Create dataframe of names with profile links and save to csv
dict = {'Names': names, 'profile_urls': profiles_final} 
       
df = pd.DataFrame(dict)
df.to_csv('profiles_final.csv') 

In [66]:
#Load if not running previous chunks
profiles = pd.read_csv('profiles_final.csv')
profiles_final = profiles['profile_urls']


In [67]:
#Create lists for each seperate field collected
i = 0
phone_list = []
title_list = []
bio_list = []
research_list = []
degree_list = []
link_list = []

#traverse through profile link file
for i in range(0,len(profiles_final)):
    #Pause to avoid bot detection
    time.sleep(3)
    #Load webpage
    driver.get(profiles_final[i])
    src = driver.page_source  
    # Now using beautiful soup
    soup = BeautifulSoup(src, 'html.parser')
    
    #Collect phone number
    person_info = soup.find_all("span", {"class": "info-staff"})
    phone = None
    for x in person_info:
        if x:
            x = x.get_text()
            if "Phone" in x:
                phone = x.replace('Phone: ', '')
                phone = phone.replace('(', '')
                phone = phone.replace(') ', '-')
    
    if phone:
        print(phone)
        phone_list.append(phone)
    else:
        phone_list.append("N/A")
    
    #Collect Job Title
    person_title = soup.find("div", {"class": "field field--name-field-org-positions field--type-text-long field--label-visually_hidden"})
    if person_title:
        person_title1 = person_title.find("p")
        title = person_title1.get_text()
        
    else:
        title = "N/A"
        
    title_list.append(title)
    
    #Collect Biography and Research
    person_bio = soup.find("div", {"class": "layout__region region-second"})
    if person_bio:
        person_degree = person_bio.find_all("li")
        temp = person_bio.get_text().split('Research\n')
        bio = temp[0].replace('Biography','')
        bio = bio.split('Degrees\n')[0]
        bio = bio.split('Honors and Awards\n')[0]
        bio = bio.replace("\n\n",'')
        #print(bio)
        if len(temp)>1:
            if "Research Interests" in temp[1]:
                res = temp[1].split("Degrees\n")[0].split("Research Interests\n")[-1]
                print(res)
            else:
                res = "N/A"
        else:
            res = "N/A"

        #Collect degrees
        degrees = ""
        for t in person_bio.find_all("li"):
            degrees = degrees + t.get_text() + "\n"
    else:
        bio = "N/A"
        degrees = "N/A"
    bio_list.append(bio)
    research_list.append(res)
    degree_list.append(degrees)
    
    
    # Personal links and email addresses
    content = soup.find("section", {"id": "content"})
    if content:
        links = ""
        for link in content.find_all('a'):
            links = links + link.get('href') + "\n"
    else:
        links = "N/A"
        
    link_list.append(links)
    
    
    i+=1

703-993-1511
Machine Learning

703-993-1624
C4I, Decision Support Systems, Human-Centered Design, Evaluation

703-909-7254
703-993-1629
Entrepreneurship and Innovation, Information Security and Assurance, Cybersecurity

703-993-1110
Artificial Intelligence, Computer Game Technology, and Computer Graphics

703-993-6158
703-993-1660
Software Engineering

703-993-5429
703-993-4383
703-993-5426
703-993-3786
Computer Systems, Parallel and Distributed Systems, Real-Time and Embedded Systems, Power-Aware Computing, Fault Tolerance

703-993-1674
703-993-6148
703-993-5313
703-993-1500
Mechanical Engineering

703-993-1627
Databases
Honors and Awards
Outstanding Researcher, CS Department,  George Mason University

703-993-5638
703-993-1606
Electronics

703-993-4381
703-993-1591
Big Data, Artificial Intelligence, Data Analytics, Personalized Learning, Educational Data Mining, Entrepreneurship and Innovation, Expert Systems, Human-Centered Design, Intelligent Agents, Machine Learning, Learning Anal

703-993-4732
703-993-1684
Classification and regression
Robust statistical inference
Biometric identification

703-993-5621
Geotechnical Engineering

703-993-1722
Data Mining, Decision Support Systems, Predictive Analytics, Artificial Intelligence, Data Analytics, Big Data, Personalized Learning, Educational Data Mining, Expert Systems, Intelligent Agents, Machine Learning, Learning Analytics, Information Security and Assurance, Knowledge Engineering, Knowledge Acquisition, Science of Evidence, Semantic Web, Cognitive Assistants

703-993-1658
703-993-5996
703-993-7176
703-993-1630
Transportation Engineering

703-993-7151
Bioinformatics and biopharmaceutical statistics
Robust inference
Branching process
Risk theory




703-993-1579
Signal and Array Processing, Underwater Acoustics, and Engineering Education.

703-993-1579
Computer and Network Security




703-993-9288
703-993-1533
Machine Learning, Artificial Intelligence, Bioinformatics, Computer Vision, Data Mining, Computer and Netwo

In [80]:
#Create Dataframe using lists storing collected inforamtion then save to a CSV file.
CEC = {'Name': profiles['Names'], 'Title': title_list, 'Phone Number': phone_list, 'Biography': bio_list, 
       'Research Interest': research_list, 'Degrees': degree_list, 'Links': link_list, 'Profile Link': profiles_final} 
       
df = pd.DataFrame(CEC)
df.fillna("N/A",inplace=True)
df['Biography'] = df['Biography'].replace(r'\n',' ', regex=True)
df['Research Interest'] = df['Research Interest'].replace(r'\n',',', regex=True)


In [81]:
#Remove errors in dataframe do to inconsistency in webpage format
for i in df['Research Interest']:
    if "Honors" in i:
        print(i)
        i = i.split("Honors")[0]
        print(i)
    if "Publications" in i:
        print(i)
        i = i.split("Publications")[0]
        print(i)



Databases,Honors and Awards,Outstanding Researcher, CS Department,  George Mason University,
Databases,
Research Interests:,Blast analysis and design of steel structural walls and roof systems,Improved blast resistance,Propagation of blast wave over roofs,Publications:,Bondok, D., Clayton, G., and Salim, H. (2018). “Dynamic Response of Roof Truss Systems under Blast Loading,” Structure Congress 2018,Bondok, D., and Salim, H. (2017). “Failure Capacities of Cold-Formed Steel Roof Trusses End-Connections,” Thin-Walled Structures Journal.,Bondok, D., Salim, H., and McClendon, M. (2017). “Static Resistance of Open Web Steel Joist Roof Systems for Blast Response Analysis,” ASCE Journal of Performance of Constructed Facilities, American Society of Civil Engineering (ASCE).,Bondok, D., Salim, H., Saucier, A., and Dinan, R. (2015). "Static Resistance Function of Cold-Formed Steel Stud Walls," Thin-Walled Structures Journal,Bondok, D., and Salim, H. (2014). “Numerical Modeling of Conventional St

In [82]:
df

,Name,Title,Phone Number,Biography,Research Interest,Degrees,Links,Profile Link
0,Sharon Adamus,"Associate Dean, Undergraduate Programs, Colleg...",703-993-1511,"Bringing the best talent, building the best pr...","Machine Learning,","PhD, Computer Science, Brown University\nScM,...",,https://cec.gmu.edu/profiles/scarabal
1,Leonard Adelman,"Professor, Systems Engineering and Operations ...",703-993-1624,Leonard Adelman’s research is oriented toward...,"C4I, Decision Support Systems, Human-Centered ...","PhD, Psychology, University of Colorado, Boul...",mailto:ladelman@gmu.edu\nhttp://seor.vse.gmu.e...,https://cec.gmu.edu/profiles/ladelman
2,Omoche Agada,Assistant Professor,N/A,Omoche Cheche Agada's areas of interests incl...,N/A,"PhD, Information Technology, George Mason Univ...",mailto:oagada@gmu.edu\n,https://cec.gmu.edu/node/8046
3,Pouyan Ahmadi,Assistant Professor,703-909-7254,Pouyan Ahmadi's research interests include coo...,N/A,"PhD, Electrical and Computer Engineering, Geo...",mailto:pahmadi@gmu.edu\n,https://cec.gmu.edu/profiles/pahmadi
4,Massimiliano Albanese,"Associate Professor, Associate Director at the...",703-993-1629,Massimiliano Albanese is an associate Profess...,"Entrepreneurship and Innovation, Information S...","PhD, Computer Science & Engineering, Universi...",mailto:malbanes@gmu.edu\nhttp://csis.gmu.edu/a...,https://cec.gmu.edu/profiles/malbanes
...,...,...,...,...,...,...,...,...
236,Qiang Zeng,Associate Professor,N/A,Dr. Qiang Zeng is an Associate Professor in t...,N/A,"PhD, Computer Science and Engineering, Penn St...",mailto:zeng@gmu.edu\nhttps://cs.gmu.edu/~zeng/\n,https://cec.gmu.edu/profiles/qzeng2
237,Linghan Zhang,"Assistant Professor, Cyber Security Engineerin...",N/A,Linghan Zhang is an Assistant Professor of th...,N/A,,mailto:lzhang37@gmu.edu\nhttps://mason.gmu.edu...,https://cec.gmu.edu/node/7096
238,Xiaokuan Zhang,Assistant Professor,N/A,Xiaokuan Zhang's research interest include co...,N/A,"PhD, Computer Science and Engineering, Ohio St...",mailto:xzhang46@gmu.edu\n,https://cec.gmu.edu/profiles/xzhang46
239,Yutao Zhong,Associate Professor\nComputer Science\nCollege...,703-993-1773,Yutao Zhong contributes to research and academ...,N/A,,mailto:yzhong@gmu.edu\nhttp://cs.gmu.edu/~yzho...,https://cec.gmu.edu/profiles/yzhong


In [83]:
#Data cleaning
#Remvoe "mailto: from beginning of email address and seperate email address from personal link"
df['Links'] = df['Links'].replace('mailto:','', regex=True)
new = df["Links"].str.split("\n", n = 1, expand = True)
df["Email"] = new[0]
df["Personal Link"] = new[1]
#Fill NaN and blanks with "N/A"
df.fillna("N/A",inplace=True)
df["Personal Link"] = df['Personal Link'].replace('','N/A', regex=True)
df["Biography"] = df['Biography'].replace('','N/A', regex=True)
df["Degrees"] = df['Degrees'].replace('','N/A', regex=True)
#Remove unnessary column
df.drop(columns =["Links"], inplace = True)
df

,Name,Title,Phone Number,Biography,Research Interest,Degrees,Profile Link,Email,Personal Link
0,Sharon Adamus,"Associate Dean, Undergraduate Programs, Colleg...",703-993-1511,"Bringing the best talent, building the best pr...","Machine Learning,","PhD, Computer Science, Brown University\nScM,...",https://cec.gmu.edu/profiles/scarabal,,N/A
1,Leonard Adelman,"Professor, Systems Engineering and Operations ...",703-993-1624,Leonard Adelman’s research is oriented toward...,"C4I, Decision Support Systems, Human-Centered ...","PhD, Psychology, University of Colorado, Boul...",https://cec.gmu.edu/profiles/ladelman,ladelman@gmu.edu,http://seor.vse.gmu.edu/faculty/adelman.html\n
2,Omoche Agada,Assistant Professor,N/A,Omoche Cheche Agada's areas of interests incl...,N/A,"PhD, Information Technology, George Mason Univ...",https://cec.gmu.edu/node/8046,oagada@gmu.edu,N/A
3,Pouyan Ahmadi,Assistant Professor,703-909-7254,Pouyan Ahmadi's research interests include coo...,N/A,"PhD, Electrical and Computer Engineering, Geo...",https://cec.gmu.edu/profiles/pahmadi,pahmadi@gmu.edu,N/A
4,Massimiliano Albanese,"Associate Professor, Associate Director at the...",703-993-1629,Massimiliano Albanese is an associate Profess...,"Entrepreneurship and Innovation, Information S...","PhD, Computer Science & Engineering, Universi...",https://cec.gmu.edu/profiles/malbanes,malbanes@gmu.edu,http://csis.gmu.edu/albanese/\n
...,...,...,...,...,...,...,...,...,...
236,Qiang Zeng,Associate Professor,N/A,Dr. Qiang Zeng is an Associate Professor in t...,N/A,"PhD, Computer Science and Engineering, Penn St...",https://cec.gmu.edu/profiles/qzeng2,zeng@gmu.edu,https://cs.gmu.edu/~zeng/\n
237,Linghan Zhang,"Assistant Professor, Cyber Security Engineerin...",N/A,Linghan Zhang is an Assistant Professor of th...,N/A,N/A,https://cec.gmu.edu/node/7096,lzhang37@gmu.edu,https://mason.gmu.edu/~lzhang37/\n
238,Xiaokuan Zhang,Assistant Professor,N/A,Xiaokuan Zhang's research interest include co...,N/A,"PhD, Computer Science and Engineering, Ohio St...",https://cec.gmu.edu/profiles/xzhang46,xzhang46@gmu.edu,N/A
239,Yutao Zhong,Associate Professor\nComputer Science\nCollege...,703-993-1773,Yutao Zhong contributes to research and academ...,N/A,N/A,https://cec.gmu.edu/profiles/yzhong,yzhong@gmu.edu,http://cs.gmu.edu/~yzhong/\n


In [84]:
#Make sure email column only contains emails and if false, move link to personal link column
for i in range(0,len(df['Email'])):
    if "http" in df['Email'][i]:
        df['Personal Link'][i] = df['Email'][i]
        df['Email'][i] = "N/A"

In [86]:
df.to_csv('profiles_final_all.csv')